In [ ]:
from collections import OrderedDict
from bs4 import BeautifulSoup

with open('4-response.html') as f:
    text = f.read()

soup = BeautifulSoup(text, 'html.parser')

d = OrderedDict()
soup

In [ ]:
soup.find_all('td')

In [10]:
import base64
import execjs
import requests


with open('src/md5.js') as f:
    jscode = f.read()

url_temp = 'http://match.yuanrenxue.com/api/match/4?page={page}'
s = execjs.compile(jscode)


def get_md5(raw):
    return s.call('hex_md5', raw)


def getlist(page):
    url = url_temp.format(page=page)
    # requests.getを使うと、レスポンス内容を取得できるのでとりあえず変数へ保存
    response = requests.get(url)

    # response.json()でJSONデータに変換して変数へ保存
    jsonData = response.json()

    return jsonData


def get_masking_class(key, value):
    s_bytes = (key + value).encode('ascii')
    base64_bytes = base64.b64encode(s_bytes)
    s_base64 = base64_bytes.decode('ascii')
    return get_md5(s_base64.replace('=', ''))

In [22]:
BeautifulSoup(getlist(1)['info'], 'lxml').find_all('td')[0].find_all('img')

[,
 ,
 ,
 ]

In [ ]:
def get_nums(html, display_filter):
    nums_bs4 = BeautifulSoup(html, 'lxml').find_all('td')
    def get_num_images(num_bs4):
        imgs_bs4 = num_bs4.find_all('img')
        imgs_bs4_display = filter(lambda img_bs4: display_filter not in img_bs4['class'], imgs_bs4)
        return [img['src'] for img in imgs_bs4_display]
    return map(get_num_images, nums_bs4)

In [ ]:
data = getlist(1)
for num in list(get_nums(data['info'], get_masking_class(data['key'], data['value']))):
    print(len(num))

In [26]:
data = getlist(1)
for num in list(get_nums(data['info'], get_masking_class(data['key'], data['value']))):
    print(len(num))

4
4
4
4
4
4
3
4
4
4
